In [3]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

import time

from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print("準備完了")


準備完了


# データを取るためにclass生成

In [4]:
class ThubanNaver:

    X_train, y_train, X_test, y_test = [], [], [], []
    index_to_word, word_to_index = {}, {}
    

    def __init__(self):
        print("생성")
        
    def set_init_words(self):
        # set word dictionary
        index_to_word = pd.read_csv('words.csv')
        self.index_to_word = {idx:word for idx, word in index_to_word['word'].items()}
        self.word_to_index = {word:idx for idx, word in self.index_to_word.items()}
        
    def set_init_datas(self):
        read_train = pd.read_csv('train.csv').dropna()
        read_test = pd.read_csv('test.csv').dropna()
        
        self.X_train = [self.get_encoded_sentence(text) for idx, text in read_train['text'].items()]
        self.X_test = [self.get_encoded_sentence(text) for idx, text in read_test['text'].items()]
        
        self.y_train = np.array([label for idx, label in read_train['label'].items()])
        self.y_test = np.array([label for idx, label in read_test['label'].items()])
        
    def save_datas_to_csv(self):
        
        # 데이터를 읽어봅시다. 
        train_txt = pd.read_table('ratings_train.txt')
        test_txt = pd.read_table('ratings_test.txt')
        # 重複のデータを削除, そしてNANを削除
        train_txt = train_txt.drop_duplicates('document').dropna()
        test_txt = test_txt.drop_duplicates('document').dropna()
        
        
        okt = Okt()
        
        stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
        
        #セーブされる配列
        train_texts = []
        test_texts = []
        clean_words = []
        

        # 単語辞書をcsvファイルにセーブために韓国語だけまたはtoken化する
        for i in range(0,2):
            if i == 0:
                read_x_datas = train_txt['document']
                save_x_datas = train_texts
            else:
                read_x_datas = test_txt['document']
                save_x_datas = test_texts
                
            for idx, item in enumerate(read_x_datas.items()):
                item_text = re.compile('[가-힣]+').findall(item[1])
                results = okt.morphs(' '.join(item_text), stem=True)
                stopwords_result = []
                for result in results:
                    if result not in stopwords: #単語たちのstopwordsを削除
                        stopwords_result.append(result)
                
                # stopwordsしても文字がある場合だけ
                if(len(stopwords_result) > 0):
                    save_x_datas.append(' '.join(stopwords_result)) # trainまたはtestデータ
                    clean_words.extend(stopwords_result) #辞書のcsvのため       
                # print("{} {}".format(results, idx))
                
                # 全部forしたら時間がかかるので（テスト用）
                #if idx == 99:
                #    break

        #単語たちの重複を削除
        clean_words = set(clean_words)
        clean_words = list(clean_words)
        
        #
        clean_words.insert(0, "<UNUSED>")
        clean_words.insert(0, "<UNK>")
        clean_words.insert(0, "<BOS>")
        clean_words.insert(0, "<PAD>")
        
        
        # words.csvセーブ
        pd.DataFrame(clean_words, columns=['word']).to_csv('words.csv', encoding="utf-8")
        self.set_init_words() #下のtexts, labelsをセーブするため (classの変数 index_to_word, word_to_indexに 実際の値を入力)
        
        
        # trainとtestをcsvでセーブする
        train_labels = [result for result in train_txt['label']][:len(train_texts)]
        test_labels = [result for result in test_txt['label']][:len(test_texts)]

        train_df = pd.DataFrame({'text':train_texts, 'label':train_labels})
        test_df = pd.DataFrame({'text':test_texts, 'label':test_labels})
        
        pd.DataFrame(train_df).to_csv('train.csv', encoding="utf-8")
        pd.DataFrame(test_df).to_csv('test.csv', encoding="utf-8")
        self.set_init_datas() # classの変数 X_train, y_train, X_test, y_testに 実際の値を入力
        
        
    def load_csv_datas(self):
        self.set_init_words()
        self.set_init_datas()
        
    def load_data(self, num_words):
        if len(self.word_to_index.items()) < num_words: #num_words数がword_to_index配列の長さより長い場合はword_to_indexの長さに従う
            num_words = len(self.word_to_index.items())
        return self.X_train, self.y_train, self.X_test, self.y_test, [ item for item, idx in self.word_to_index.items() if idx < int(num_words) ]
        
        
    # 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
    # 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
    def get_encoded_sentence(self, sentence):
        return [self.word_to_index['<BOS>']]+[self.word_to_index[word] if word in self.word_to_index else self.word_to_index['<UNK>'] for word in sentence.split()]

    # 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
    def get_encoded_sentences(self, sentences):
        return [self.get_encoded_sentence(sentence) for sentence in sentences]
    
    # 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
    def get_decoded_sentence(self, encoded_sentence):
        return ' '.join(self.index_to_word[index] if index in self.index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

    # 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
    def get_decoded_sentences(self, encoded_sentences):
        return [self.get_decoded_sentence(encoded_sentence) for encoded_sentence in encoded_sentences]

TN = ThubanNaver()

생성


'ratings_train.txt'と'ratings_test.txt'だけで   
X_train, y_train, X_test, y_test, words_to_indexの値を得るためのclass

# データ加工

In [5]:
print("スタート")
start = time.time()
TN.save_datas_to_csv()
print("データセーブ、{}秒でロード完了".format(time.time()-start))

スタート
データセーブ、368.9126796722412秒でロード完了


In [ ]:
# すでにTN.save_datas_to_csv()をして words.csv, x_test.csv, x_train.csv ファイルが居る場合は下の関数を利用してデータを読み込みする
TN.set_init_words()
TN.set_init_datas()
print("データロード完了")


データを初めて扱っているならTN.save_datas_to_csv()関数を   
そうじゃなければなければ   
TN.set_init_words()   
TN.set_init_datas()   
を使ってください   
   
### こうやって二つのコードに割った理由は TN.save_datas_to_csv() この関数が結構時間がかかるため

# データ獲得

In [6]:
X_train, y_train, X_test, y_test, words_to_index = TN.load_data(num_words=113661)
print("훈련 샘플 개수: {}, 테스트 개수: {}".format(len(X_train), len(X_test)))
print("単語数: {}".format(len(words_to_index)))

훈련 샘플 개수: 145233, 테스트 개수: 48776
単語数: 47999


load_data(num_words=n)のnum_wordsで単語を何個持ってくるのかを決めます   
存在する単語数より多い場合は存在する単語すべてを持ってきます

# 配列の長さをお互い同じくする

In [7]:
total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  11.838930152724874
문장길이 최대 :  72
문장길이 표준편차 :  9.036148727026863
pad_sequences maxlen :  29
전체 문장의 0.932889711302053%가 maxlen 설정값 이내에 포함됩니다. 


# 配列の長さをお互い同じくする

In [8]:
X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=TN.word_to_index["<PAD>"],
                                                        padding='pre', # 혹은 'pre'
                                                        maxlen=maxlen)

X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=TN.word_to_index["<PAD>"],
                                                       padding='pre', # 혹은 'pre'
                                                       maxlen=maxlen)

print("clear")

clear


配列の長さをお互い同じくしなければ   
モデルの訓練ができなくなります   
   
そしてpad_sequencesの関数の中でpadding='pre'はpreがいいらしい   
X_train、X_testの配列一つ一つには文章があるんですが個々長さが違うので後ろのには空白が出ます   
モデルの訓練の時空白が最後に計算して悪い影響が出るそうです   
なので列の前の方に空白を置いて後ろは実際の文字を置くpreがいいらしいです

In [9]:
print(X_train.shape)

(145233, 29)


# モデル作成または訓練

In [10]:
vocab_size = len(words_to_index)    # 어휘 사전의 크기입니다
word_vector_dim = 100  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 



model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model.add(keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경가능)
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])





#검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면 학습을 조기 종료(Early Stopping)합니다. 
#또한, ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#에포크는 총 15번을 수행하겠습니다. 또한 훈련 데이터 중 20%를 검증 데이터로 사용하면서 정확도를 확인합니다.
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         4799900   
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 3488      
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 4,803,469
Trainable params: 4,803,469
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
1933/1937 [============================>.] - ETA: 0s - loss: 0.6932 - acc: 0.5019
Epoch 00001: val_acc improved from -inf to 0.49499, saving model to best_model.h5
1937/1937 [==============================] - 20s 10ms/step - loss: 0.

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

正確度がひくくってこのまま使うには無理そうです。   
なのであるブログで見た韓国語トークン化をもっとよくして正確度を上げるつもりです   

# 単語の数を減らす

二度以下しか出ない単語も含めて訓練すれば良くないらしいです   
なのでその単語は削除します

In [11]:
X_train, y_train, X_test, y_test, words_to_index = TN.load_data(num_words=113661)
print("훈련 샘플 개수: {}, 테스트 개수: {}".format(len(X_train), len(X_test)))
print("単語数: {}".format(len(words_to_index)))

훈련 샘플 개수: 145233, 테스트 개수: 48776
単語数: 47999


In [12]:
X_train_dec = TN.get_decoded_sentences(X_train)
X_test_dec = TN.get_decoded_sentences(X_test)
print("clear")

clear


X_trainはいまvetorなので韓国語に戻す

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_dec)
print(tokenizer.word_index)


{'영화': 1, '보다': 2, '을': 3, '없다': 4, '이다': 5, '있다': 6, '좋다': 7, '너무': 8, '다': 9, '정말': 10, '되다': 11, '재밌다': 12, '적': 13, '만': 14, '같다': 15, '진짜': 16, '로': 17, '점': 18, '아니다': 19, '않다': 20, '에서': 21, '만들다': 22, '연기': 23, '나오다': 24, '것': 25, '평점': 26, '내': 27, '최고': 28, '그': 29, '나': 30, '안': 31, '인': 32, '스토리': 33, '생각': 34, '못': 35, '왜': 36, '드라마': 37, '게': 38, '감동': 39, '사람': 40, '보고': 41, '이렇다': 42, '말': 43, '고': 44, '아깝다': 45, '더': 46, '아': 47, '때': 48, '배우': 49, '거': 50, '감독': 51, '그냥': 52, '재미있다': 53, '요': 54, '내용': 55, '시간': 56, '재미': 57, '중': 58, '뭐': 59, '까지': 60, '주다': 61, '자다': 62, '하고': 63, '지루하다': 64, '재미없다': 65, '수': 66, '쓰레기': 67, '가다': 68, '네': 69, '모르다': 70, '들다': 71, '그렇다': 72, '싶다': 73, '작품': 74, '지': 75, '사랑': 76, '알다': 77, '다시': 78, '하나': 79, '마지막': 80, '볼': 81, '이건': 82, '오다': 83, '정도': 84, '저': 85, '완전': 86, '많다': 87, '처음': 88, '장면': 89, '액션': 90, '주인공': 91, '차다': 92, '걸': 93, '이렇게': 94, '안되다': 95, '개': 96, '나다': 97, '지금': 98, '돈': 99, '최악': 100, '이야기': 101, '하': 1

tokenizer.fit_on_texts()関数を使ってX_train_decの中で一番出現が高い単語の順番で順位を付けて並ぶ

In [14]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 42410
등장 빈도가 2번 이하인 희귀 단어의 수: 23346
단어 집합에서 희귀 단어의 비율: 55.04833765621315
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8329280552558396


In [15]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 19066


三度以上出現した単語の数

In [16]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train_dec)
X_train = tokenizer.texts_to_sequences(X_train_dec)
X_test = tokenizer.texts_to_sequences(X_test_dec)
print("clear")

clear


順番が変わったword_to_indexによって韓国語を数値化する

In [35]:
index_to_word={word:index for index, word in tokenizer.word_index.items()}

In [40]:
# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences):
    return [get_decoded_sentence(encoded_sentence) for encoded_sentence in encoded_sentences]

Tokenizer()によってword_index、index_to_wordが変わったので再び正義します

In [42]:
print(X_train[:3])
print(get_decoded_sentences(X_train[:3]))

[[48, 444, 17, 256, 647], [896, 445, 42, 596, 2, 210, 1430, 24, 943, 667, 21], [367, 2401, 1, 4966, 6189, 3, 217, 10]]
['더빙 진짜 짜증나다 목소리', '포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다', '무재 OOV 다그 래서 보다 추천 다']


’OOV’はニかい以下出現した単語です

In [44]:
total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))


X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=TN.word_to_index["<PAD>"],
                                                        padding='pre', # 혹은 'pre'
                                                        maxlen=maxlen)

X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=TN.word_to_index["<PAD>"],
                                                       padding='pre', # 혹은 'pre'
                                                       maxlen=maxlen)

print("clear")

문장길이 평균 :  10.838930152724874
문장길이 최대 :  71
문장길이 표준편차 :  9.036148727026863
pad_sequences maxlen :  28
전체 문장의 0.932889711302053%가 maxlen 설정값 이내에 포함됩니다. 
clear


In [50]:
print(X_train[0])
print(X_train[1])
print(len(X_train[0]))
print(len(X_train[1]))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0  48 444  17 256 647]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0  896  445   42  596    2  210 1430   24  943  667   21]
28
28


配列の長さを一致する

In [ ]:
word_vector_dim = 16  # 단어 하나를 표현하는 임베딩 벡터의 차원수입니다. 

# 리뷰 분류를 위해서 LSTM을 사용합니다.
model = Sequential()
model.add(Embedding(vocab_size, word_vector_dim))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

#검증 데이터 손실(val_loss)이 증가하면, 과적합 징후므로 검증 데이터 손실이 4회 증가하면 학습을 조기 종료(Early Stopping)합니다. 
#또한, ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장합니다.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

#에포크는 총 15번을 수행하겠습니다. 또한 훈련 데이터 중 20%를 검증 데이터로 사용하면서 정확도를 확인합니다.
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)


#해당 모델 출처 https://wikidocs.net/44249

In [60]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1525/1525 [==============================] - 2s 1ms/step - loss: 0.7243 - acc: 0.5008

 테스트 정확도: 0.5008


これでもダメだった

# 한국어 Word2Vec 임베딩 활용하여 성능개선

これがあると思えなかったです   
遅い感があるけどやってみます   
https://github.com/Kyubyong/wordvectors

In [61]:
from gensim.models import KeyedVectors
word2vec_path = 'ko.tar.gz'
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=True, limit=1000000)
# vector = word2vec['computer']
# vector     # 무려 300dim의 워드 벡터입니다.

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 512: invalid continuation byte